In [1]:
# import statements

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os 
from skimage.color import rgb2gray
import tensorflow as tf
import matplotlib.pyplot as plt

In [7]:
#functions

# this function reads and image with the directory file_path, resizes the image, and converts it to grayscale
# @param file_path is the directory of the image
def ResizeConvert(img):
    """resizes image, converts to grayscale"""
    assert type(img) != type(None)
    newImg = cv2.resize(img, (28, 28))
    assert newImg.shape == (28, 28, 3)
    gray_image = cv2.cvtColor(newImg, cv2.COLOR_BGR2GRAY)
    assert gray_image.shape == (28, 28)
    return gray_image

# this function uses directoryName to find the list of images in that directory. 
# It then uses the function ReadResizeConvert to change the images 
# then it appends the images to an array already created outside the function.
# @param directoryName is a string that is part of the directory
#def imageToArray(directoryName, localFinalImagesArr, localFinalAnswersArr, answer):
#    listOfImages = os.listdir(directoryName)
#    directory = directoryName + '/'
#    for i in listOfImages:
#        pathOfImage = directory + i
#        img = cv2.imread(pathOfImage)
#        output = ResizeConvert(img)
#        localFinalImagesArr.append(output)
#        localFinalAnswersArr.append(answer)
        
def videoToArray(videoName, localFinalImagesArr, localFinalAnswersArr, answer):
    cam = cv2.VideoCapture(videoName)
    while True:
        ret, frame = cam.read()
        if frame is None:
            break
        output = ResizeConvert(frame)
        localFinalImagesArr.append(output)
        localFinalAnswersArr.append(answer)
    print ('videoToArray is returning ', len(localFinalImagesArr), ' samples')
        
        
# takes raw image arrays after conversion and modifies it so that the model can use it        
def prepareData(data):
    data = data.reshape(data.shape[0], 28, 28, 1)
    data = data.astype('float32')
    data /= 255
    return data

In [12]:
#finalImagesArr is the array in which we put the images from the array data1 that we keep adding to
#finalAnswersArr contains the corresponding labels for the images in myData

finalImagesArrTrain = []
finalAnswersArrTrain = []

#imageToArray('/Users/23amrutad/Desktop/cup-original', finalImagesArrTrain, finalAnswersArrTrain, 0)
#imageToArray('/Users/23amrutad/Desktop/coke-original', finalImagesArrTrain, finalAnswersArrTrain, 1)


videoToArray('/Users/23amrutad/Desktop/training_videos/cup.MOV', finalImagesArrTrain, finalAnswersArrTrain, 0)
videoToArray('/Users/23amrutad/Desktop/training_videos/coke.MOV', finalImagesArrTrain, finalAnswersArrTrain, 1)

x_train = np.array(finalImagesArrTrain)
y_train = np.array(finalAnswersArrTrain)

finalImagesArrTest = []
finalAnswersArrTest = []

#imageToArray('/Users/23amrutad/Desktop/cup_test', finalImagesArrTest, finalAnswersArrTest, 0)
#imageToArray('/Users/23amrutad/Desktop/cup-sunny', finalImagesArrTest, finalAnswersArrTest, 0)
#imageToArray('/Users/23amrutad/Desktop/coke_test', finalImagesArrTest, finalAnswersArrTest, 1)
#imageToArray('/Users/23amrutad/Desktop/coke-sunny', finalImagesArrTest, finalAnswersArrTest, 1)

videoToArray('/Users/23amrutad/Desktop/training_videos/cup_test.MOV', finalImagesArrTest, finalAnswersArrTest, 0)
videoToArray('/Users/23amrutad/Desktop/training_videos/coke_test.MOV', finalImagesArrTest, finalAnswersArrTest, 1)

x_test = np.array(finalImagesArrTest)
y_test = np.array(finalAnswersArrTest)

videoToArray is returning  304  samples
videoToArray is returning  684  samples
videoToArray is returning  314  samples
videoToArray is returning  667  samples


In [13]:
#this code makes a list of arrays with entries for x_train and the corresponding y_train
#it changes the list to an np array, shuffles the array, 
#and puts the x_train and y_train, now shuffled, back in their respective training arrays
#we have to shuffle them together because if we shuffle them separately, it goes out of sync

xy_list = [];
count = 0;
for _ in x_train:
    myArr = [x_train[count], y_train[count]]
    xy_list.append(myArr)
    count+=1;
xy_nparr = np.array(xy_list)
np.random.shuffle(xy_nparr)
print (xy_nparr.shape)
count = 0;
for _ in xy_nparr:
    x_train[count] = xy_nparr[count][0]
    y_train[count] = xy_nparr[count][1]
    count += 1;

(684, 2)


In [14]:
if False:
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print (x_train.shape)
    print (y_train.shape)
else:
    x_train = prepareData(x_train)
    print (x_train.shape)
    x_test = prepareData(x_test)
    print (x_test.shape)
 
input_shape = (28, 28, 1)

 
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
 

 
model = Sequential()
model.add(Conv2D(28, kernel_size=(3,3), input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(16, activation=tf.nn.relu))
model.add(Dropout(0.5))
model.add(Dense(2, activation=tf.nn.softmax))
 
 
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x=x_train,y=y_train, epochs=50)
model.evaluate(x_test, y_test)

(684, 28, 28, 1)
(667, 28, 28, 1)
Epoch 1/50
684/684 [==============================] - 1s 761us/step - loss: 0.6758 - accuracy: 0.5760
Epoch 2/50
684/684 [==============================] - 0s 336us/step - loss: 0.6397 - accuracy: 0.6901
Epoch 3/50
684/684 [==============================] - 0s 329us/step - loss: 0.6139 - accuracy: 0.6915
Epoch 4/50
684/684 [==============================] - 0s 307us/step - loss: 0.6077 - accuracy: 0.6871
Epoch 5/50
684/684 [==============================] - 0s 303us/step - loss: 0.6047 - accuracy: 0.6974
Epoch 6/50
684/684 [==============================] - 0s 307us/step - loss: 0.6045 - accuracy: 0.7120
Epoch 7/50
684/684 [==============================] - 0s 292us/step - loss: 0.6020 - accuracy: 0.6857
Epoch 8/50
684/684 [==============================] - 0s 326us/step - loss: 0.5953 - accuracy: 0.6959
Epoch 9/50
684/684 [==============================] - 0s 528us/step - loss: 0.5944 - accuracy: 0.6974
Epoch 10/50
684/684 [===========================

[0.486675848935855, 0.830584704875946]

In [17]:
cam = cv2.VideoCapture(0)
import time
while True:
    localFinalImagesArr = []
    ret, frame = cam.read()
    if frame is None:
        break
    output = ResizeConvert(frame)
    localFinalImagesArr.append(output)
    x_live = np.array(localFinalImagesArr)
    x_live = prepareData(x_live)
    prediction = model.predict(x_live)
    print (prediction)
    time.sleep(1)
    #if prediction[0, 0] > prediction[0, 1]:
        #print(0)
    #else:
        #print (1)

[[1.000000e+00 3.664563e-09]]
[[1.0000000e+00 5.1703863e-09]]
[[1.0000000e+00 9.9795905e-09]]
[[1.0000000e+00 1.0929164e-08]]
[[1.0000000e+00 7.0475887e-09]]
[[1.000000e+00 5.309439e-09]]
[[1.0000000e+00 2.6459226e-09]]
[[1.000000e+00 8.376869e-09]]
[[1.0000000e+00 3.3564482e-09]]
[[9.9999988e-01 1.5406559e-07]]
[[9.9999881e-01 1.1997122e-06]]
[[9.9999893e-01 1.0498783e-06]]
[[1.0000000e+00 1.2163028e-08]]
[[9.999945e-01 5.475654e-06]]
[[9.9999690e-01 3.0532183e-06]]
[[9.999974e-01 2.677612e-06]]
[[9.9999976e-01 2.0565510e-07]]
[[9.999999e-01 7.439355e-08]]
[[9.999976e-01 2.429585e-06]]
[[9.999956e-01 4.445698e-06]]
[[9.9999976e-01 2.3452907e-07]]
[[9.9999940e-01 5.7867567e-07]]
[[9.9999976e-01 2.0798961e-07]]
[[9.9999964e-01 4.0976462e-07]]
[[9.9999940e-01 6.2943457e-07]]
[[9.999994e-01 6.544229e-07]]
[[1.000000e+00 1.597659e-09]]
[[9.999993e-01 7.573980e-07]]
[[1.0000000e+00 3.3830842e-12]]
[[1.0000000e+00 1.6730736e-10]]
[[1.0000000e+00 4.1801402e-09]]
[[1.0000000e+00 4.0578336e-09]

KeyboardInterrupt: 